In [275]:
import os
import re
import sys
import math
import time
import errno

from PIL import Image as PImage
import win32com.client as win32

from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5 import uic

from pdf2image import convert_from_path
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Border, Side
from openpyxl.worksheet.page import PageMargins
from openpyxl.worksheet.pagebreak import Break


In [276]:
#------------------------------------------------ 소제목 체크 ---------------------------------------------------

def title_check(sentence):

    # 중분류(항목명) 필터링
    if sentence[0:2] == '7.' and sentence[2:sentence.find('.',2)].isnumeric() and sentence[sentence.find('.',2):sentence.find('.',2)+2] == '. ': return True
    return False

def title_check1(sentence):
    # 시험장소 필터링
    if sentence[0:8] == '나. 시험 장소': return True
    return False    

def title_check2(sentence):
    # 시험장소 필터링
    if sentence[0:13] == '라. 의뢰자 제시 기준': return True
    return False    

def title_check3(sentence):
    # 시험장소 필터링
    if sentence[0:11] == '7.2. 시험 결과': return True
    return False
    


In [277]:
#------------------------------------------------ TXT 기준 소제목 추출 ---------------------------------------------------

def title_extract(txt_path):

    # 해당 경로의 텍스트 파일 데이터 읽기
    try:
        with open(txt_path, 'r', encoding = 'cp949') as f:
            data = f.read()
    except: 
        with open(txt_path, 'r', encoding = 'utf8') as f:
            data = f.read()

    # 데이터 전처리(여러 줄내림_CRLF 삭제)
    for i in range(10):
        data = data.replace('\n\n','\n')
    for i in range(10):
        data = data.replace('\n'+' '*i+'\n','\n')  

    # 줄내림_CRLF 문자 기준으로 Split
    data = data.split('\n')
    
    # 필터링된 항목명
    item = []
    # 해당 항목명 시작 페이지 번호
    pages = []
    pg_count=0
    # 필터링된 목표값
    goal = []
    # 필터링된 단위
    unit = []
    # 필터링된 결과값
    result = []
    
    # split된 데이터 순회(줄 문장들을 확인함)
    for index, sentence in enumerate(data):
            
        # 항목명 발견시 페이지 번호/ 항목명 결과 삽입
        if title_check(sentence):
            pages.append(pg_count)
            item.append(sentence)
            
        # 시험장소 발견시 인덱스 저장
        if title_check1(sentence):
            placeIdx = index

        # 의뢰자 제시 기준 발견시 인덱스 저장
        if title_check2(sentence):
            goalIdx = index

        # 시험 결과 발견시 인덱스 저장
        if title_check3(sentence):
            resultIdx = index
        
        # 가장 변경되지 않을 것으로 보이는 양식명을 기준으로 페이지를 카운트(!!! 양식 업데이트시 변경 요망 !!!)
        if sentence == '양식TQP-12-01-03(1)':
            pg_count = pg_count+1
        #if sentence == 'forsplit(!*@&#^$%)':

    
    place = data[placeIdx+1].lstrip()
    
    for i in range(len(item)) :
        if i < 3 :
            goal.append(' ')
            unit.append(' ')
            result.append(' ')
        else :
            goal.append(data[goalIdx-5+(i*4)])
            #print('goal',data[goalIdx+7+(i*4)],goalIdx,i,goalIdx+7+(i*4))
            unit.append(data[goalIdx-4+(i*4)])
            result.append(data[resultIdx-5+(i*4)])
            
    # 이미지 기능 통합을 위해 마지막 CAP 추가
    item.append(' --END--')
    pages.append(pg_count)
                      
    return item, pages, place, goal, unit, result


In [278]:
#------------------------------------------------ HWP 파일 변환 함수 ---------------------------------------------------

def hwp2etc(inout_path, output_path,option='TEXT'):
    
    # 오류 회피
    if inout_path == None: return 0
    if output_path == None: return 0
    #if option not in ('TEXT','PDF'): return 0
    
    # HWP 객체 Open 및 Option으로 저장
    hwp = win32.gencache.EnsureDispatch('HWPFrame.HwpObject')
    hwp.XHwpWindows.Item(0).Visible = False
    hwp.RegisterModule('FilePathCheckDLL', 'FilePathCheckerModule')
    hwp.Open(inout_path)
    hwp.FindCtrl()
    hwp.SaveAs(output_path,option)
    hwp.Clear(1)
    hwp.Quit()
    
    # 변환 성공시 1 리턴
    return 1



In [279]:
#------------------------------------------------ PDF 파일 이미지 변환 함수 ---------------------------------------------------

def pdf2img(image_path):
    
    images = convert_from_path(image_path, 
                               fmt='jpg', 
                               dpi=600, 
                               #first_page=0, 
                               #last_page=9, 
                               poppler_path=r"Release-23.07.0-0\poppler-23.07.0\Library\bin")

    dir_path = os.path.dirname(image_path) + '/page_images'

    try:
        os.makedirs(dir_path)
    except OSError as exc: 
        if exc.errno == errno.EEXIST and os.path.isdir(dir_path):
            pass

    for idx, img in enumerate(images):
        img.save(dir_path + '/page' + str(idx) + '.jpg')

    return dir_path

#----------------------------------------------------- 스레드 클래스 통합 ----------------------------------------------------

In [280]:
#------------------------------------------------ 이미지 크롭 ---------------------------------------------------

def img_crop(image_path):
    img_list = os.listdir(os.path.dirname(image_path)+'/page_images/')
    for i in range(len(img_list)):
        path = os.path.dirname(image_path)+'/page_images/page'+str(i)+'.jpg'
        img = PImage.open(path)
        img = img.crop((200,1000,4759,6030))
        img = img.resize((539,595))
        img.save(os.path.dirname(image_path)+'/page_images/page'+str(i)+'.jpg')
        
#----------------------------------------------------- 스레드 클래스 통합 ----------------------------------------------------

In [281]:
#------------------------------------------------ 엑셀에 굵은선 네모 ---------------------------------------------------

def excel_rectangle(sht, start_point, end_point):
    
    # 워크시트 받아오기
    ws = sht
    
    # 좌상단 꼭짓점 열번호
    xs = start_point[1]
    # 우하단 꼭짓점 열번호
    xe = end_point[1]
    # 좌상단 꼭짓점 행번호
    ys = start_point[0]
    # 우하단 꼭짓점 행번호
    ye = end_point[0]
    
    # 열 이동하면서 가로 줄 그림
    for i in range(xe - xs +1):
        ws.cell(ys,i+xs).border = Border(top=Side(border_style='thick',color='000000'))
        ws.cell(ye,i+xs).border = Border(bottom=Side(border_style='thick',color='000000'))

    # 행 이동하면서 세로 줄 그림
    for i in range(ye - ys + 1):
        ws.cell(i+ys,xs).border = Border(left=Side(border_style='thick',color='000000'))
        ws.cell(i+ys,xe).border = Border(right=Side(border_style='thick',color='000000'))

    # 각 꼭짓점 4개 그림
    ws.cell(ys, xs).border = Border(top=Side(border_style='thick',color='000000'),left=Side(border_style='thick',color='000000'))
    ws.cell(ye, xs).border = Border(bottom=Side(border_style='thick',color='000000'),left=Side(border_style='thick',color='000000'))
    ws.cell(ys, xe).border = Border(top=Side(border_style='thick',color='000000'),right=Side(border_style='thick',color='000000'))
    ws.cell(ye, xe).border = Border(bottom=Side(border_style='thick',color='000000'),right=Side(border_style='thick',color='000000'))



In [282]:
#------------------------------------------------ HWP -> PDF 스레드 ---------------------------------------------------

class HWP_Thread(QThread):
    
    # 시그널 오브젝트 생성
    data_signal = pyqtSignal(object)

    # 초기화
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
    
    # 스레드 
    def run(self):
        
        # Caller path 가져오기{path1:한글파일 경로, path2:작업공간 경로}
        file_path = self.parent.path1
        file_name = os.path.basename(file_path)
        
        # Review_[] 작업 공간 생성
        dir_name = self.parent.path2 + '/Review_' + file_name.replace('.hwp','')
        progress_string1 = ''
        try:
            os.makedirs(dir_name)
            progress_string1 = '10%'+'Review_' + file_name.replace('.hwp','') + ' Directory has been Made...'
        except OSError as exc: 
            if exc.errno == errno.EEXIST and os.path.isdir(dir_name):
                progress_string1 = '10%'+'Review_' + file_name.replace('.hwp','') + ' Directory already been Made...'
        # 생성 시그널 메인 윈도우 전송
        self.data_signal.emit(progress_string1)
        
        # 생성할 파일들 지정
        txt_path = dir_name + '/' + file_name.replace('.hwp','.txt')
        pdf_path = dir_name + '/' + file_name.replace('.hwp','.pdf')
        
        # TXT 파일로 변환하기
        start = time.time()
        state = hwp2etc(file_path, txt_path)
        end = time.time()
        progress_string2 = '40%'+'Translate ' + file_name + ' to txt file, elapsed ' + str(round(end - start,5))+' s...'
        # 변환 시그널 메인 윈도우 전송
        self.data_signal.emit(progress_string2)
        
        # PDF 파일로 변환하기
        start = time.time()
        state = hwp2etc(file_path, pdf_path, 'PDF')
        end = time.time()
        # 변환 시그널 메인 윈도우 전송
        progress_string3 = '100%'+'Translate ' + file_name + ' to pdf file, elapsed ' + str(round(end - start,5))+' s...'
        self.data_signal.emit(progress_string3)
        
        # 종료 시그널 메인 윈도우 전송
        self.msleep(1000)
        self.data_signal.emit('close_form%'+pdf_path)


In [283]:
class PDF_Thread(QThread):
    
    # 시그널 오브젝트 생성
    data_signal = pyqtSignal(object)

    # 초기화
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        
    # 스레드
    def run(self):
        
        # 시작 시그널 메인 윈도우 전송
        progress_string0 = '1%'+ self.parent.path1 + ' Loaded..'
        self.data_signal.emit(progress_string0)
        
        # PDF 파일 이미지 분리 
        start = time.time()
        images = convert_from_path(self.parent.path1, 
                               fmt='jpg', 
                               dpi=600, 
                               poppler_path=r"Release-23.07.0-0\poppler-23.07.0\Library\bin")
        end = time.time()
        
        # 이미지 분리 완료 시그널 메인 전송
        progress_string1 = '30%'+ self.parent.path1 + ' Seperated to jpg files, elapsed ' + str(round(end - start,5))+' s..'
        self.data_signal.emit(progress_string1)
        
        # 분리 이미지 저장 공간 확보
        progress_string2 = ''
        dir_path = os.path.dirname(self.parent.path1) + '/page_images'
        try:
            os.makedirs(dir_path)
            progress_string2 = '50%'+ dir_path + ' has been Made..'
        except OSError as exc: 
            if exc.errno == errno.EEXIST and os.path.isdir(dir_path):
                progress_string2 = '50%'+ dir_path + ' already been Made..'
        
        # 저장 공간 확보 시그널 메인 전송
        self.msleep(300)
        self.data_signal.emit(progress_string2)
        
        # 저장 공간에 이미지 저장
        totals = len(images)
        for idx, img in enumerate(images):
            start = time.time()
            img.save(dir_path + '/page' + str(idx) + '.jpg')
            end = time.time()
            
            # 작업률 계산해서 시그널 전송
            percents = int(idx/totals*20+50)
            progress_string3 = str(percents)+'%Number '+ str(idx) + ' image Saved elapsed ' + str(round(end - start,5))+' s..'
            self.data_signal.emit(progress_string3)
            
        # 크롭할 이미지 확인
        img_list = os.listdir(os.path.dirname(dir_path)+'/page_images/')
        totals = len(img_list)
        
        # 이미지 고정 사이즈로 크롭
        for i in range(len(img_list)):
            start = time.time()
            path = os.path.dirname(dir_path)+'/page_images/page'+str(i)+'.jpg'
            img = PImage.open(path)
            img = img.crop((200,1000,4759,6030))
            img = img.resize((539,595))
            img.save(os.path.dirname(dir_path)+'/page_images/page'+str(i)+'.jpg')
            end = time.time()
            
            # 작업률 계산해서 시그널 전송
            percents = int(i/totals*30+70)
            progress_string4 = str(percents)+'%Number '+ str(i) + ' image Croped elapsed ' + str(round(end - start,5))+' s..'
            self.data_signal.emit(progress_string4)
        
        # 종료 시그널 메인 윈도우 전송
        self.msleep(1000)
        self.data_signal.emit('close_form%'+dir_path)


In [290]:
class MyStatus(QDialog):
    
    # 클래스 변수 초기값 지정
    option = 0
    path1 = ''
    path2 = ''
    result = ''
    
    # 초기화
    def __init__(self, option=0, path1='', path2=''):
        super().__init__()
        # ui 지정, 타이틀, 아이콘 지정
        self.ui = uic.loadUi('Status_View.ui', self)
        self.setWindowTitle('KCL_분석노트_생성기 v1.1.0')
        self.setWindowIcon(QIcon('icon.png'))
        
        if option==2:
            self.setWindowFlags(Qt.WindowStaysOnTopHint)
        
        # 생성자 값 받아오기
        self.option = option
        self.path1 = path1
        self.path2 = path2
        
        # 스테이터스 창 띄우기
        self.show()
        
        # option 1 = HWP -> PDF
        if self.option == 1:
            self.label.setText('HWP to PDF')
            self.hwpload()  
        # option 1 = PDF -> JPG
        elif self.option == 2:
            self.label.setText('PDF to JPGs')
            self.pdfload()
        
    # HWP -> PDF Fuction
    def hwpload(self):
        # 스레드 클래스 생성 및 시그널 함수 지정
        x = HWP_Thread(self)
        x.data_signal.connect(self.on_data_ready)
        x.start()
        
    # PDF -> JPG Function
    def pdfload(self):
        # 스레드 클래스 생성 및 시그널 함수 지정
        x = PDF_Thread(self)
        x.data_signal.connect(self.on_data_ready)
        x.start()

    # 시그널 통신 함수
    def on_data_ready(self, data):
        
        # 프로그레스바 % 수신
        pgs = data[0:data.find('%')]
        
        # 종료 명령일 경우 스테이터스창 닫기
        if pgs == 'close_form':
            self.result = data[data.find('%')+1:]
            self.close()
            return 
        
        # 프로그레스바 % 설정, 로그에 메시지 표시
        self.listWidget.addItem(data[data.find('%')+1:])
        self.listWidget.setCurrentRow(self.listWidget.count()-1)
        self.progressBar.setValue(int(data[0:data.find('%')]))


In [288]:
#------------------------------------------------ GUI 코드 시작 ---------------------------------------------------

# ui 지정
form_class = uic.loadUiType('KCL_testnote_maker.ui')[0]

#------------------------------------------------ UI 연결 및 함수 연결 ---------------------------------------------------
class MyWindow(QMainWindow, form_class):
    
    # 초기화
    def __init__(self):
        super().__init__()
        
        # ui 셋업, 타이틀 지정, 아이콘 지정
        self.setupUi(self)              
        self.setWindowTitle('KCL_분석노트_생성기 v1.1.0')
        self.setWindowIcon(QIcon('icon.png'))
        
        # 현재 시간 설정
        self.currentDateTime = QDateTime.currentDateTime()
        self.dateTimeEdit.setDateTime(self.currentDateTime)
        
        # 버튼 함수 지정
        self.Button_sel_hwp.clicked.connect(self.sel_hwp)
        self.Button_sel_dir.clicked.connect(self.sel_dir)
        self.Button_hwp2pdf.clicked.connect(self.hwp_load_thread)
        self.Button_pdf2jpg.clicked.connect(self.pdf_load_thread)
        self.Button_prs_hwp.clicked.connect(self.parsing)
        self.Button_sel_xls.clicked.connect(self.sel_xls)
        self.Button_mk_xls.clicked.connect(self.mk_xlsx)
        self.Button_help.clicked.connect(self.help_function)
        
    # hwp 로드 함수
    def hwp_load_thread(self):
        # hwp 에러처리
        if not os.path.isfile(self.Line_sel_hwp.text()):
            QMessageBox.warning(self,'Warning','성적서 을지(hwp)를 선택해주세요!')
            return
        
        # dir 에러처리
        dir_path = self.Line_sel_dir.text()
        if not os.path.isdir(dir_path):
            dir_path = os.path.dirname(self.Line_sel_hwp.text())
        
        # 투명도 낮추고 스테이터스 창 호출(option = 1 : HWP -> PDF)
        self.setWindowOpacity(0.8)
        self.second = MyStatus(1, self.Line_sel_hwp.text(), dir_path)
        self.second.exec()
        # 결과물(생성된 pdf 경로) 수신 후 투명도 복구
        self.Line_tr_hwp.setText(self.second.result)
        self.setWindowOpacity(1.0)
        
    # pdf 분리 함수
    def pdf_load_thread(self):
        # pdf 혹시 모를 에러처리
        if self.Line_tr_hwp.text() == '':
            QMessageBox.warning(self,'Warning','성적서 을지를 변환(pdf) 해주세요!')
            return 
        if not os.path.isfile(self.Line_tr_hwp.text()):
            QMessageBox.critical(self,'Failure','PDF 경로에 문제가 있습니다!')
            return
        
        # 투명도 낮추고 스테이터스 창 호출(option = 2 : HWP -> JPG)
        self.setWindowOpacity(0.8)
        self.second = MyStatus(2, self.Line_tr_hwp.text())
        self.second.exec()
        # 결과물(분리된 JPG 경로) 수신 후 투명도 복구
        self.Line_tr_pdf.setText(self.second.result)
        self.setWindowOpacity(1.0)

    # 한글 을지 선택 함수
    def sel_hwp(self):
        # 작업공간 에러처리(지정되어 있으면 키고 아니면 디폴트)
        dir_path = self.Line_sel_dir.text()
        if not os.path.isdir(dir_path):
            dir_path = ''
        # 파일 다이얼로그 호출    
        fname = QFileDialog.getOpenFileName(self, '변환할 한글파일 선택',dir_path, filter='*.hwp')
        self.Line_sel_hwp.setText(fname[0])
        
    # 엑셀 원본 선택 함수
    def sel_xls(self):
        # 작업공간 지정
        dir_path = self.Line_sel_dir.text()
        if not os.path.isdir(dir_path):
            dir_path = os.path.dirname(self.Line_sel_hwp.text())
            
        # 파일 다이얼로그 호출
        fname = QFileDialog.getOpenFileName(self, '원본 엑셀파일 선택',dir_path, filter='*.xlsx')
        self.Line_sel_xls.setText(fname[0])
        
    # 작업공간 선택 함수
    def sel_dir(self):
        # 디렉토리 다이얼로그 호출
        folder = QFileDialog.getExistingDirectory(self, '저장 디렉토리 선택')
        self.Line_sel_dir.setText(folder)
    
    # 항목 추출 함수
    def parsing(self):
        
        # 파싱 에러처리
        if self.Line_tr_hwp.text() == '':
            QMessageBox.warning(self,'Warning','성적서 을지를 변환(pdf) 해주세요!')
            return 
        if not os.path.isfile(self.Line_tr_hwp.text().replace('.pdf','.txt')):
            QMessageBox.critical(self,'Failure','TXT 경로에 문제가 있습니다!')
            return
        
        start = time.time()
        # 항목 추출
        item, pages, place, goal, unit, result = title_extract(self.Line_tr_hwp.text().replace('.pdf','.txt'))
        
        # 테이블에 표시
        omission = 2
        self.tableWidget.setRowCount(len(result)-2)
        self.tableWidget.setColumnCount(7)
        self.tableWidget.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        for i in range(len(result)-omission):
            self.tableWidget.setItem(i,0,QTableWidgetItem(item[i+omission]))
            self.tableWidget.setItem(i,1,QTableWidgetItem(item[i+omission][item[i+omission].find(' ')+1:]))
            self.tableWidget.setItem(i,2,QTableWidgetItem(goal[i+omission]))
            self.tableWidget.setItem(i,3,QTableWidgetItem(unit[i+omission]))
            self.tableWidget.setItem(i,4,QTableWidgetItem(result[i+omission]))
            self.tableWidget.setItem(i,5,QTableWidgetItem(str(pages[i+omission])))
            self.tableWidget.setItem(i,6,QTableWidgetItem(place))
        end = time.time() 
        self.Line_prs_hwp.setText(self.Line_tr_hwp.text().replace('.pdf','.txt'))
    
    # 노트 생성 함수
    def mk_xlsx(self):
        
        # 예외 처리
        if not os.path.isfile(self.Line_sel_xls.text()):
            QMessageBox.warning(self,'Warning','분석노트 원본(xlsx)를 선택해주세요!')
            return
        
        # 작업 공간
        dir_path = self.Line_sel_dir.text()
        if not os.path.isdir(dir_path):
            dir_path = os.path.dirname(self.Line_sel_hwp.text())
        
        # 테이블에서 데이터 받아오기
        data = []
        for i in range(self.tableWidget.rowCount()):
            tmp = []
            tmp.append(self.tableWidget.item(i, 1).text())
            tmp.append(self.tableWidget.item(i, 2).text())
            tmp.append(self.tableWidget.item(i, 3).text())
            tmp.append(self.tableWidget.item(i, 4).text())
            tmp.append(int(self.tableWidget.item(i, 5).text()))
            tmp.append(self.tableWidget.item(i, 6).text())
            data.append(tmp)

        #try:
        # 엑셀 연결
        load_wb = load_workbook(self.Line_sel_xls.text())
        # 시트 연결
        ws = load_wb['결과']
        ws['C2'].value = '소프트웨어 시험'
        point_row = 5
        point_data = ws['B'+str(point_row)].value
        while point_data != None :
            ws['H'+str(point_row)].value = '-'
            ws['I'+str(point_row)].value = ws['F'+str(point_row)].value
            ws['K'+str(point_row)].value = data[point_row-4][2]
            ws['L'+str(point_row)].value = data[point_row-4][3].replace(data[point_row-4][2],'')
            ws['M'+str(point_row)].value = data[point_row-4][1]
            ws['N'+str(point_row)].value = '-'
            ws['O'+str(point_row)].value = 'N'
            ws['P'+str(point_row)].value = data[point_row-4][5]
            ws['Q'+str(point_row)].value = 'N'
            point_row = point_row + 1
            point_data = ws['B'+str(point_row)].value

        ws['B'+str(point_row)].value = '* 시험평가 전 시험용 장비의 초기화 진행 여부 확인'
        ws.print_area = 'B1:Q'+str(point_row)
        ws.page_margins = PageMargins(left=0.5, right=0.5, top=0.5, bottom=0.5)
        ws.page_setup.orientation = 'landscape'
        ws.page_setup.scale = 50

        ws = load_wb['부가정보']
        ws['C2'].value = '소프트웨어 시험'
        ws.print_area = 'B1:I6'
        ws.page_margins = PageMargins(left=0.5, right=0.5, top=0.5, bottom=0.5)
        ws.page_setup.orientation = 'landscape'
        ws.page_setup.scale = 83
        
        ws = load_wb['분석내용']

        # 시험 환경 네모 그리기
        excel_rectangle(ws, (3,1),(7,9))
        ws.unmerge_cells('A3:B3')
        ws.unmerge_cells('G3:H3')
        ws.merge_cells('A3:I7')
        ws.cell(row=3,column=1, value='시험환경')
        point_row = 9
        page_break_rows = []
        for inx in range(len(data)-1):

            # 시험환경이 아닌 항목들 네모 그리고 제목 설정
            if inx != 0:
                ws['A'+str(point_row+1)].value = '(' + str(inx) + ') 항목명'
                ws['G'+str(point_row+1)].value = '작업기준'
                ws['G'+str(point_row+2)].value = '의뢰자 제시방법'
                ws['A'+str(point_row+2)].value = data[inx][0]
                excel_rectangle(ws, (point_row,1),(point_row+3,9))
                point_row = point_row + 5

            # jpg 붙이기
            for jnx in range(data[inx+1][4] - data[inx][4]):
                ws['A'+str(point_row)].value = 'page'+str(data[inx][4]+jnx)
                tmp_path = os.path.dirname(self.Line_tr_hwp.text())+'/page_images'
                image_path = tmp_path + '/page'+str(data[inx][4]+jnx)+'.jpg'
                image = Image(image_path)
                ws.add_image(image, 'A'+str(point_row))
                point_row = point_row + 28
                page_break_rows.append(point_row-1)

        for row in page_break_rows:
            ws.row_breaks.append(Break(id=row))
        
        # 뒤에 mk 붙여서 다른이름 저장
        new_name = os.path.dirname(self.Line_sel_xls.text())+'/' + os.path.basename(self.Line_sel_xls.text()).replace('.xlsx','_mk.xlsx')
        load_wb.save(new_name)

        # 경로 표시 및 알림
        self.Line_mk_xls.setText(new_name)
        QMessageBox.information(self,'Info','분석노트 생성이 완료되었습니다!')
            
        #except: 
            #QMessageBox.critical(self,'Failure','생성 중 오류가 발생했습니다. \n 선택된 파일들을 확인해주세요')
            
    def help_function(self):
        info_string = ''
        info_string += '제작자 : 소프트웨어평가센터 백성준\n\n'
        info_string += '버  전 : KCL_Testnote_Maker v1.1.0\n\n'
        info_string += '문의처 : staro199@kcl.re.kr\n\n'
        info_string += '필수SW : MS Excel, 한글, Poppler\n\n'
        info_string += '구성품 : Main_ui, Status_ui, icon \n\n'
        info_string += '상  세 : 파이썬 3.7, PyQt5, Win32, Openpyxl, PIL, Pdf2image\n\n\n'
        info_string += '한글 접근 오류 영구적 해결 방법_[https://martinii.fun/67]\n '
        info_string += '레지스트리 값은 "FilePathCheckerModule" 설정 요망 \n\n'
        
        info_string += '함께 배포된 Release 폴더에 Poppler는 경로가 없어지면 고장납니다.'
        QMessageBox.information(self,'Info',info_string)



In [289]:
if __name__ == '__main__':
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
        
    #font, ok = QFontDialog.getFont()
    app.setFont(QFont('경기천년제목 Light', 12))
    
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()